In [1]:
!pip install darts -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 592.0/592.0 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 KB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 KB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.4/716.4 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.4/372.4 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 KB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build 

# Import de bibliotecas

In [2]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 

from datetime import datetime
from itertools import product

from unicodedata import normalize

from darts import TimeSeries, concatenate
from darts.datasets import AustralianTourismDataset
from darts.models import LinearRegressionModel, Theta
from darts.metrics import mae
from darts.dataprocessing.transformers import MinTReconciliator
from darts.dataprocessing import Pipeline
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis
from darts.utils.timeseries_generation import linear_timeseries
from darts.models.forecasting.xgboost import XGBModel
import time
from darts.metrics import rmse
from darts.models.forecasting.random_forest import RandomForest
from darts.models import  AutoARIMA
from darts.metrics import mape

from darts.metrics import coefficient_of_variation as coef_var


# Carregamento de dados

In [3]:
def salvar_arquivo(file_to_save,kind:str):
    PATH = f'/content/drive/MyDrive/Dados/standarized/{kind}/' + kind + '_' + datetime.strftime(datetime.today(), '%d-%m-%Y')+'.csv'
    file_to_save.to_csv(PATH, index=False)

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Dados/unified-data/full_table_17022023.csv')

# Filtragem de dados:

Produto|Variavel
-------|--------
Abacaxi|Produto das lavouras temporárias
Amendoim|Produto das lavouras temporárias
Batata-Doce|Produto das lavouras temporárias
Feijão|Produto das lavouras temporárias
Melancia|Produto das lavouras temporárias
Milho|Produto das lavouras temporárias
Tomate|Produto das lavouras temporárias
Soja|Produto das lavouras temporárias
Bovino|Pecuária


## Lavouras temporárias

Todos os produtos que serão analisados no artigo estão listados aqui. A suas área de coberturas estão classificadas como "Outros", entretanto, será modificado para "Cobertura".

In [5]:
# Produtos de lavouras temporárias
prod_lavouras_temp = ['Abacaxi*', 'Amendoim (em casca)', 'Batata-doce', 'Cebola'
                    'Feijão (em grão)', 'Melancia', 'Milho (em grão)', 'Tomate']

list_concat_temp = []

for i in prod_lavouras_temp:
    df_temp = df.loc[df['produto'] == i]
    list_concat_temp.append(df_temp)

df_lavouras_temp = pd.concat(list_concat_temp)

del list_concat_temp

df_lavouras_temp

,municipio,cod_municipality,regiao_intermediaria,cod_regiao_intermediaria,cod_regiao_imediata,regiao_imediata,tipo_producao,cod_variavel,variavel,produto,metrica,valor,ano
240,Abaetetuba,1500107,Abaetetuba,150003,1501,Belém,Produto das lavouras temporárias,214.0,Quantidade produzida,Abacaxi*,Toneladas,0.0,1974.0
241,Abaetetuba,1500107,Abaetetuba,150003,1501,Belém,Produto das lavouras temporárias,214.0,Quantidade produzida,Abacaxi*,Toneladas,0.0,1975.0
242,Abaetetuba,1500107,Abaetetuba,150003,1501,Belém,Produto das lavouras temporárias,214.0,Quantidade produzida,Abacaxi*,Toneladas,6.0,1976.0
243,Abaetetuba,1500107,Abaetetuba,150003,1501,Belém,Produto das lavouras temporárias,214.0,Quantidade produzida,Abacaxi*,Toneladas,3.0,1977.0
244,Abaetetuba,1500107,Abaetetuba,150003,1501,Belém,Produto das lavouras temporárias,214.0,Quantidade produzida,Abacaxi*,Toneladas,16.0,1978.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
247783,Xinguara,1508407,Xinguara,150014,1504,Redenção,Produto das lavouras temporárias,214.0,Quantidade produzida,Tomate,Toneladas,0.0,2017.0
247784,Xinguara,1508407,Xinguara,150014,1504,Redenção,Produto das lavouras temporárias,214.0,Quantidade produzida,Tomate,Toneladas,0.0,2018.0
247785,Xinguara,1508407,Xinguara,150014,1504,Redenção,Produto das lavouras temporárias,214.0,Quantidade produzida,Tomate,Toneladas,0.0,2019.0
247786,Xinguara,1508407,Xinguara,150014,1504,Redenção,Produto das lavouras temporárias,214.0,Quantidade produzida,Tomate,Toneladas,0.0,2020.0


Modificação de produto para "Cobertura".

In [6]:
df_cobertura_lavoura_temp = df.loc[(df['produto'] == 'Outros') & (df['tipo_producao'] == 'Produto das lavouras temporárias')]
df_cobertura_lavoura_temp['produto'] = 'Cobertura'

<ipython-input-6-99035454a237>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cobertura_lavoura_temp['produto'] = 'Cobertura'


## Soja

In [7]:
df_soja = df.loc[df['produto'] == 'Soja (em grão)']

## Pastagem/Bovino

In [8]:
df_bovino = df.loc[df['produto'] == 'Bovino']
df_pasto = df.loc[df['produto'] == 'Pasto']

# Unificação de dados

Tabelas unificadas:

```
[df_bovino, df_cobertura_lavoura_temp, df_lavouras_temp, df_pasto, df_soja]
```

Produtos exatraídos:

Produto|
-------|
Bovino |
Cobertura |
Abacaxi* |
Amendoim (em casca) |
Batata-doce |
Melancia |
Milho (em grão) |
Tomate |
Pasto |
Soja (em grão) |

In [9]:
df2 = pd.concat([df_bovino, df_pasto])
df2['estado'] = 'Pará'
for i in df2.produto.unique():
    print(i,'|')

df2.head()

Bovino |
Pasto |


,municipio,cod_municipality,regiao_intermediaria,cod_regiao_intermediaria,cod_regiao_imediata,regiao_imediata,tipo_producao,cod_variavel,variavel,produto,metrica,valor,ano,estado
0,Abaetetuba,1500107,Abaetetuba,150003,1501,Belém,Pecuária,105.0,Efetivo dos rebanhos,Bovino,Cabeças,28.0,1974.0,Pará
1,Abaetetuba,1500107,Abaetetuba,150003,1501,Belém,Pecuária,105.0,Efetivo dos rebanhos,Bovino,Cabeças,54.0,1975.0,Pará
2,Abaetetuba,1500107,Abaetetuba,150003,1501,Belém,Pecuária,105.0,Efetivo dos rebanhos,Bovino,Cabeças,47.0,1976.0,Pará
3,Abaetetuba,1500107,Abaetetuba,150003,1501,Belém,Pecuária,105.0,Efetivo dos rebanhos,Bovino,Cabeças,39.0,1977.0,Pará
4,Abaetetuba,1500107,Abaetetuba,150003,1501,Belém,Pecuária,105.0,Efetivo dos rebanhos,Bovino,Cabeças,10.0,1978.0,Pará


# Limpeza/Modelagem

In [10]:
df_model = df2[['municipio', 'regiao_intermediaria', 'regiao_imediata',
                'tipo_producao', 'variavel', 'produto', 'metrica', 'ano', 'valor']]

Agrupamento por ano

In [11]:
geo_df = pd.read_excel('/content/drive/MyDrive/Dados/dados-de-apoio/Regioes_Geograficas_IBGE_Composicao_por_municipios.xls', 
                       sheet_name='PA').fillna(method='ffill')

geo_df.columns = ['cod_mun', 'uf', 'municipio', 'reg_inter', 'nome_mun_reg_inter',
                  'reg_imed', 'nome_mun_reg_imed']

geo_df = geo_df.drop(columns = ['nome_mun_reg_imed', 'nome_mun_reg_inter'])

df_model = df_model[['municipio', 'tipo_producao', 'variavel', 'produto', 'metrica',
                     'ano', 'valor']]

print(df_model.shape)                     
df_merge = df_model.merge(geo_df, on='municipio',how='inner')


df_merge = df_merge.loc[(df_merge['ano'] >=1985)]
df_ano = df_merge.groupby('ano', as_index=False).sum()[['ano']].astype('int')


(13017, 7)


## Modelagem

In [12]:
df_final = df_ano[['ano']]

drop_columns = list()

dict_plot = dict()
list_plot = list()

list_var = list()



for variav in df_merge.variavel.unique():
    list_prod = list()
    df_temp1 = df_merge.loc[df_merge['variavel'] == variav]
    v_1 = df_temp1.groupby('ano', as_index=False).sum()['valor'].sum()

    
    if v_1 > 1000: 
        # 2.1.1 Normaliza item e armazena em x2
        x_0 = normalize('NFKD', variav).encode('ASCII','ignore').decode('ASCII').upper().replace(' ', '_')
        x_0 = 'VAR_' + x_0
        df_final[x_0] = df_temp1.groupby('ano', as_index=False).sum()['valor']

        # Anexar as variáveis 
        
        list_var.append(x_0)
        

        for prod in df_temp1.produto.unique():
            
            print(prod)
            df_temp2 = df_temp1.loc[df_temp1['produto'] == prod]
            v_2 = df_temp2.groupby('ano', as_index=False).sum()['valor'].sum()
            

            if v_2 > 1000: 
                
                # 2.1.1 Normaliza item e armazena em x2
                x_01 = normalize('NFKD', prod).encode('ASCII','ignore').decode('ASCII').upper().replace(' ','_').replace('(', '').replace(')','')
                x_01 = x_0 + ' - PRODUTO_' + x_01
                list_prod.append(x_01)
                
                # 2.1.4 Agrupa, soma e salva o somatório de regi~pes imediatas em df_final
                df_final[x_01] = df_temp2.groupby('ano', as_index=False).sum()['valor']
                
                
                list_inter = list()              
                for i_inter in df_temp2['reg_inter'].unique():
                    df_temp3 = df_temp2.loc[df_temp2['reg_inter'] == i_inter]
                    v_inter = df_temp3.loc[df_temp3['reg_inter'] == i_inter].groupby('ano', as_index=False).sum()['valor'].sum() 
                    print(v_inter)
                    if v_inter > 1000: 
                        # 1.2.1 Normaliza e colocar em capital, amazena em {x}
                        x = normalize('NFKD', i_inter).encode('ASCII','ignore').decode('ASCII').upper()
                        
                        x = x_01 + ' - REGINTER_' + x
                        list_inter.append(x)
                        
                        
                                # 1.2.2 Sobreescreve x de 1.2.1 com o préfixo de {INTER_} + {x}
                            # 1.2.3 Retorna para df_final salvando na coluna o somatório de soja das regiões intermediárias
                            #       onde i_inter = Regiões intemediárias & x = nome das colunas no df_final{Início}
                            
                        df_final[x] = df_temp3.loc[df_temp3['reg_inter'] == i_inter].groupby('ano', as_index=False).sum()['valor']
                            # 1.2.4 Registra temporariamente em df_temp os registros com base na região
                            #       intermediária descrita em 1.2
                        
                        

                        # ----------------------------- Região Imediata --------------------------------
                        list_imed = list()
                        # 2.1 Para cada {i_imed} em {df_temp}{1.4.2}
                        for i_imed in df_temp3['reg_imed'].unique():
                            df_temp4 = df_temp3.loc[df_temp3['reg_imed'] == i_imed]
                            v_imed = df_temp4.groupby('ano', as_index=False).sum()['valor'].sum()
                            print(v_imed)
                            if v_imed > 1000: 
                                # 2.1.1 Normaliza item e armazena em x2
                                x2 = normalize('NFKD', i_imed).encode('ASCII','ignore').decode('ASCII').upper().replace(' ', '').replace('-','')
                                x2 = x + ' - REGIMED_' + x2
                                list_imed.append(x2)
                                # 2.1.4 Agrupa, soma e salva o somatório de regi~pes imediatas em df_final
                                df_final[x2] = df_temp4.groupby('ano', as_index=False).sum()['valor']
                            else:
                                drop_columns.append(x2)

                        dict_plot[x] = list_imed
                dict_plot[x_01] = list_inter                    
        dict_plot[x_0] = list_prod                         
dict_plot['TOTAL'] = list_var


for i in df_final.columns:
    if df_final[i].sum() == 0:
        df_final = df_final.drop(columns = [i])

df_final['TOTAL'] = df_final[list_var].sum(axis=1)

Bovino
11812227.0
4650512.0
6957266.0
204449.0
137097688.0
78491494.0
37249252.0
21356942.0
16587122.0
5835643.0
10751479.0
178689941.0
47216034.0
75659400.0
55814507.0
58275440.0
19027314.0
28063052.0
11185074.0
41113755.0
1803055.0
39310700.0
47276986.0
6108924.0
26970004.0
2159715.0
4450517.0
7587826.0
Pasto
29335980.84068218
10977779.708820365
15520557.169717312
2837643.9621444996
146336308.86018997
82347777.28388868
42746116.77890296
21242414.79739835
4005673.1685987096
3746519.0324938092
259154.1361049017
177088171.6909273
40976919.05256693
93380002.70305422
42731249.93530615
77382812.81648104
34660271.99560356
32553049.24138064
10169491.579496825
46879679.74215111
5918157.871467346
40961521.87068376
91303455.64170465
13576462.766291538
40324925.55093424
9498563.71485911
12735175.911715515
15168327.697904233


In [13]:
df_final = df_final + (0.00001* np.random.rand(df_final.shape[0], df_final.shape[1]))
df_final['ano'] = df_ano['ano']

In [14]:
df_final['TOTAL'] = df_final[list_var].sum(axis=1)

In [15]:
df_final

,ano,VAR_EFETIVO_DOS_REBANHOS,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_ABAETETUBA,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_BELEM,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_CAMETA,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA - REGIMED_MARABA,VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA - REGIMED_TUCURUI,...,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_ALTAMIRA,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_ALTAMIRA - REGIMED_ALMEIRIMPORTODEMOZ,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_ALTAMIRA - REGIMED_ALTAMIRA,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_CASTANHAL,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_CASTANHAL - REGIMED_BRAGANCA,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_CASTANHAL - REGIMED_PARAGOMINAS,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_CASTANHAL - REGIMED_CAPANEMA,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_CASTANHAL - REGIMED_CAPITAOPOCO,VAR_AREA_DE_COBERTURA - PRODUTO_PASTO - REGINTER_CASTANHAL - REGIMED_CASTANHAL,TOTAL
0,1985,3.446825e+06,3.446825e+06,142483.000002,37341.000009,102579.000001,2563.000004,6.499120e+05,5.968720e+05,5.304000e+04,...,4.425032e+05,159379.353141,2.831238e+05,1.490995e+06,178439.113037,7.490575e+05,136034.611441,199569.909045,227894.276254,1.022053e+07
1,1986,4.102298e+06,4.102298e+06,192805.000001,73342.000002,116217.000004,3246.000008,6.517330e+05,5.785600e+05,7.317300e+04,...,4.129635e+05,142983.100454,2.699804e+05,1.465069e+06,178332.039268,7.472007e+05,119419.537356,199242.179689,220874.713708,1.068974e+07
2,1987,4.641462e+06,4.641462e+06,211405.000007,76995.000007,129179.000000,5231.000008,7.926560e+05,7.147550e+05,7.790100e+04,...,4.248910e+05,116114.893582,3.087761e+05,1.588392e+06,182486.759709,7.893329e+05,133618.495077,214312.782889,268640.864449,1.176622e+07
3,1988,5.431061e+06,5.431061e+06,232784.000001,96694.000002,130997.000007,5093.000009,7.806910e+05,6.963000e+05,8.439100e+04,...,4.091791e+05,89350.653630,3.198285e+05,1.722176e+06,203419.592140,8.237939e+05,157913.011811,235400.105078,301649.152426,1.307823e+07
4,1989,5.955474e+06,5.955474e+06,278257.000007,143607.000003,129457.000007,5193.000006,1.291445e+06,8.563690e+05,2.889760e+05,...,4.087488e+05,70753.193144,3.379956e+05,1.666691e+06,185340.019879,8.049924e+05,162745.528502,222275.173544,291338.314749,1.376186e+07
5,1990,6.292421e+06,6.292421e+06,282747.000006,148540.000005,129242.000006,4965.000010,1.363288e+06,9.089880e+05,2.983000e+05,...,4.000584e+05,50572.330112,3.494861e+05,1.691122e+06,185094.461502,8.313490e+05,152781.188757,242129.942067,279767.198853,1.442838e+07
6,1991,6.714444e+06,6.714444e+06,285850.000005,151700.000008,129589.000008,4561.000003,1.372751e+06,9.172610e+05,3.016900e+05,...,3.919395e+05,48913.072442,3.430264e+05,1.838052e+06,279786.011889,8.369256e+05,164244.943602,261676.954122,295418.334107,1.521271e+07
7,1992,7.066588e+06,7.066588e+06,316478.000004,151450.000010,161022.000000,4006.000007,1.489186e+06,1.001462e+06,3.265240e+05,...,4.505395e+05,68006.704105,3.825328e+05,1.929894e+06,274843.050911,8.739700e+05,194495.450556,259280.383148,327304.970720,1.627016e+07
8,1993,7.515580e+06,7.515580e+06,371707.000009,153500.000004,214028.000010,4179.000004,1.403395e+06,8.837900e+05,3.896050e+05,...,5.221311e+05,99036.892309,4.230942e+05,1.966172e+06,273750.739202,8.922308e+05,204137.280247,255781.599575,340272.052520,1.736278e+07
9,1994,7.624234e+06,7.624234e+06,392198.000007,158530.000010,229320.000005,4348.000005,1.393568e+06,8.832800e+05,3.721880e+05,...,6.026589e+05,158657.703271,4.440012e+05,1.924824e+06,273635.769406,8.999090e+05,190930.306195,251322.873172,309026.389915,1.795518e+07


In [16]:
dict_plot

{'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM': ['VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_ABAETETUBA',
  'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_BELEM',
  'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_CAMETA'],
 'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA': ['VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA - REGIMED_MARABA',
  'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA - REGIMED_TUCURUI',
  'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA - REGIMED_PARAUAPEBAS'],
 'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BREVES': ['VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BREVES - REGIMED_BREVES',
  'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BREVES - REGIMED_SOURESALVATERRA'],
 'VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_REDENCAO': ['VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_REDENCAO - REGIME

In [17]:
print(f'{"="*50}\nNOME DE COLUNAS\n{"="*50}\n\n')
for i in df_final.columns:
    print(i,'\n-----------------------')
    print(df_final[i].sum(),'\n')

NOME DE COLUNAS


ano 
-----------------------
74111 

VAR_EFETIVO_DOS_REBANHOS 
-----------------------
490853159.00018334 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO 
-----------------------
490853159.00017923 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM 
-----------------------
11812227.000185873 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_ABAETETUBA 
-----------------------
4650512.000194839 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_BELEM 
-----------------------
6957266.000154845 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_BELEM - REGIMED_CAMETA 
-----------------------
204449.00020585745 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA 
-----------------------
137097688.0001631 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA - REGIMED_MARABA 
-----------------------
78491494.00019045 

VAR_EFETIVO_DOS_REBANHOS - PRODUTO_BOVINO - REGINTER_MARABA - REGIMED_TUCURUI 
--------

In [18]:
print(f'{"="*50}\nNOME DE COLUNAS = 0 \n{"="*50}\n\n')
for i in df_final.columns:
    if df_final[i].sum() == 0:
        print(i,'\n-----------------------')
        print(df_final[i].sum(),'\n')


NOME DE COLUNAS = 0 




In [19]:
salvar_arquivo(df_final, 'time_series')

# Criação de hierarquias

In [20]:
hierarchy = dict()
# Itens que não devem ser iterados (totais)

# Para cada Item nas colunas:
for i in df_final.columns:
    # Condicional
    if i != 'ano' and i != 'TOTAL':

        # Text_split = i (nome da coluna) separado por " - "
        text_split = i.split(' - ')
        # Lenf é igual ao tamanho do texto separado (nível hierarquico)
        lenf = len(text_split)
        # Nova lista que irá conter as suas hierarquias superiores
        new_list = [] 

        # Enquanto o contador (tamanho do texto separado) for maior que 1:
        while lenf > 1:
            # Debugger
            new_list.append(' - '.join(text_split[0:lenf-1]))
            lenf-= 1
        hierarchy[i] = new_list
        
keys = []
for k, v in hierarchy.items():
    if len(v) == 0:
        hierarchy[k] = ['TOTAL']

for i in keys:
    if i not in df_final.columns:
        print(i)

In [21]:
with open(f'/content/drive/MyDrive/Dados/standarized/dict_plot/dict_plot_{datetime.strftime(datetime.today(), "%d-%m-%Y")}.pkl', 'wb') as fp:
    pickle.dump(dict_plot, fp)


In [22]:
with open(f'/content/drive/MyDrive/Dados/standarized/hierarchy/hierarchy_{datetime.strftime(datetime.today(), "%d-%m-%Y")}.pkl', 'wb') as fp:
    pickle.dump(hierarchy, fp)


In [23]:
time_series = TimeSeries.from_dataframe(df =df_final, time_col= 'ano', value_cols=df_final.columns.drop('ano'), hierarchy=hierarchy)

In [24]:
time_series

<TimeSeries (DataArray) (ano: 37, component: 61, sample: 1)>
array([[[ 3446825.00000707],
        [ 3446825.00000529],
        [  142483.00000232],
        ...,
        [  199569.90904528],
        [  227894.27625445],
        [10220525.51853197]],

       [[ 4102298.00000334],
        [ 4102298.00000404],
        [  192805.00000064],
        ...,
        [  199242.17968858],
        [  220874.71370776],
        [10689740.45323532]],

       [[ 4641462.00000826],
        [ 4641462.00000536],
        [  211405.00000715],
        ...,
...
        ...,
        [  400289.75026784],
        [  522039.41882134],
        [42152166.66724842]],

       [[22473948.00000318],
        [22473948.00000038],
        [  364142.00000573],
        ...,
        [  393523.74112748],
        [  520379.14672206],
        [43784624.0592017 ]],

       [[23964405.00000222],
        [23964405.0000024 ],
        [  380961.00000095],
        ...,
        [  386913.03137083],
        [  512944.50207021],
        [46046033.17560604]]])
Coordinates:
  * ano        (ano) int64 1985 1986 1987 1988 1989 ... 2017 2018 2019 2020 2021
  * component  (component) object 'VAR_EFETIVO_DOS_REBANHOS' ... 'TOTAL'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          {'VAR_EFETIVO_DOS_REBANHOS': ['TOTAL'], 'VAR_EFETIVO_...